This is a code in an unofficial walkthrough of the Expectation Maximization algorithm in Figure 1 of

Do, C. B., & Batzoglou, S. (2008). What is the expectation maximization algorithm? Nature Biotechnology, 26(8), 897–899. http://doi.org/10.1038/nbt1406

I wanted to better understand what exactly the expectation maximization algorithm was doing at an intuitive level, as most of the abstract and purely mathematical explanations I had watched on youtube or read online weren't clicking. I've always found there is no better way to learn something than to try it until it works and makes sense.

If you have access to the original paper, I highly recommend reading it, as I'm sure this will make much more sense after reading the authors original words and explanation

![](header.png)

And, as mentioned earlier, we will focus on the lone figure of the paper, and work through the iterations step by step. Here is the full figure. Try and walk through it mentally, and see which parts you can understand directly from the figure, and make a mental note of which steps don't make sense. During the walk through, come back to these original ideas and see which parts you were correct on, and see if the code helps the unclear steps make sense

![](Figure1_with_caption.png)

In [1]:
import pandas as pd

### The Observed Data

Here is the table from Figure 1 that shows the true underlying data.
![](coin_table.png)

If we could observe which coin was used for each round, we could compute the true probability of each coin
![](true_data.png)

However, in the event we are unable to know which coin was used in each trial, we would like to approximate the maximum likelihood probabilities of the two coins

Let's set up a DataFrame in Julia that represents this

In [3]:
Data = {
    'COIN' : ['B', 'A', 'A', 'B', 'A'],
    'FLIPS' : ['HTTTHHTHTH', 'HHHHTHHHHH', 'HTHHHHHTHH', 'HTHTTTHHTT', 'THHHTHHHTH']
}
Data = pd.DataFrame(Data)
Data

,COIN,FLIPS
0,B,HTTTHHTHTH
1,A,HHHHTHHHHH
2,A,HTHHHHHTHH
3,B,HTHTTTHHTT
4,A,THHHTHHHTH


We have included the true state of the coin, however, we will blind ourselves to that knowledge and proceed as if we could not observe true state of each coin used. We can however observe the resulting flip data. Let's go ahead and precompute the number of heads and tails for each trial for easier calculations later. I'll break down the steps first though.

The countmap function takes an iterable, like a list, and counts the number of occurances of each unique value in the iterable.

Our trials are currently in the form of a string

In [4]:
Data['FLIPS'][0]

'HTTTHHTHTH'

Now let's turn it into a list by taking a list comprehension on it.

The ' operator on the end is the transpose operator. It simply coverts the array from long, columnar form to wide, row form and makes it cleaner to print to the console

In [5]:
[a for a in range(4)]

[0, 1, 2, 3]

In [6]:
[flip for flip in Data['FLIPS'][0]]

['H', 'T', 'T', 'T', 'H', 'H', 'T', 'H', 'T', 'H']

now we can wrap that in a countmap call to get a dictionary of counts for heads and tails

In [7]:
from collections import Counter

In [8]:
test = Counter([flip for flip in Data['FLIPS'][0]])
print(test)
list1 = dict(Counter([flip for flip in Data['FLIPS'][0]]))
print(Data['FLIPS'])
list1['H']

Counter({'H': 5, 'T': 5})
0    HTTTHHTHTH
1    HHHHTHHHHH
2    HTHHHHHTHH
3    HTHTTTHHTT
4    THHHTHHHTH
Name: FLIPS, dtype: object


5

In [9]:
l1=[]
for i in range(len(Data['FLIPS'])):
#     l1.append(Data['FLIPS'][i])
    l1.append(dict(Counter([flip for flip in Data['FLIPS'][i]]))['H'])
print(l1)

[5, 9, 8, 4, 7]


and to get the numbers of heads and tails, we just need to do a dictionary look up with `['H']` and `['T']`

now we are ready to do the same steps as above, but use the map function to perform the calculations on every trial in the experiment, which we can do with the map function

In [10]:
l2=[]
for i in range(len(Data['FLIPS'])):
#     l2.append(Data['FLIPS'][i])
    l2.append(dict(Counter([flip for flip in Data['FLIPS'][i]]))['T'])
print(l2)

[5, 1, 2, 6, 3]


In [11]:
Data['HEADS'] = l1
Data['TAILS'] = l2

Here is the Data with precomupted heads and tails

In [12]:
Data

,COIN,FLIPS,HEADS,TAILS
0,B,HTTTHHTHTH,5,5
1,A,HHHHTHHHHH,9,1
2,A,HTHHHHHTHH,8,2
3,B,HTHTTTHHTT,4,6
4,A,THHHTHHHTH,7,3


Here are the values from the figure for comparison

![](heads_tails.png)

For a santiy check, let's compute the true heads probabilities of each of the coins to confirm that they match what is shown in the figure. To do this, we will compute the probabiities by computing over grouped DataFrames

Here we group the DataFrame `Data` by the values in the column `COIN`, and then on each of the resulting sub DataFrames (one each for coin A and coin B), we sum the total number of heads of all trials for the given coin, and then divide that by the total number of flips for the given coin to calculate the head probability for each coin. Again, confirm this matches the probabilities in the figure

In [13]:
print(Data.loc[Data['COIN']=='A']['HEADS'])
res=[]
res.append(sum(Data.loc[Data['COIN']=='A']['HEADS'])/(sum(Data.loc[Data['COIN']=='A']['HEADS'])+sum(Data.loc[Data['COIN']=='A']['TAILS'])))
res.append(sum(Data.loc[Data['COIN']=='B']['HEADS'])/(sum(Data.loc[Data['COIN']=='B']['HEADS'])+sum(Data.loc[Data['COIN']=='B']['TAILS'])))
res

1    9
2    8
4    7
Name: HEADS, dtype: int64


[0.8, 0.45]

In [14]:
# by(Data, :COIN, df -> sum(df[:HEADS]) / (sum(df[:HEADS]) + sum(df[:TAILS])))

![](original_data.png)

Great! Now we are set up to start the algorithm. Before we can start, we need to choose initial values for the probabilties. Do & Batzoglou 2008 used $θ_A = 0.6$ and $θ_B = 0.5$

![](initial_probabilities.png)

We could however, choose any initial starting values. We will follow the example in the paper first. Later, I will try and use randomly drawn initial values to see if we converge to the same values

In [15]:
θA_init = 0.6;
θB_init = 0.5;

Now that we have our initial states and our data, we can now evaluate the relative probabilities that each trial of 10 coin flips was generated by coin A or coin B.

Again, assuming that the trails were produced by an unknown coin, our data now looks like this
![](unassigned_coins.png)

We want to assign a relative probability to each of those trials being generated by each of the possible states (coins)

Here is our now unlabeled data
![](trial_1.png)

And let's see how we can grab the precomputed # of heads

In [16]:
Data.loc[0, ['HEADS']]

HEADS    5
Name: 0, dtype: object

and tails

In [17]:
da=Data.loc[0, ['TAILS']]
print(da.values)

[5]


Now we can think of total number of flips as being a generic total number of trials. In the same manner, we can think of the number of heads as being a generic number of successes. 

In [18]:
successes = Data.loc[0, ['HEADS']].values
successes = int(successes)
trials = Data.loc[0, ['HEADS']].values + Data.loc[0, ['TAILS']].values
trials = int(trials)
print(successes, trials, 2**3)

5 10 8


In this format, we can now plug the data into a generic binomial likelihood formula to calculate the likelihood of observing the given data.

The formula is

$$ {n \choose k}\theta^k (1-\theta)^{n-k} $$

where
$$ n = \text{# trails} $$
$$ k = \text{# successes} $$
$$ \theta = \text{likelihood of getting a heads for a given coin} $$

In [19]:
# θA
import math
theta = θA_init
C = lambda n,k: math.factorial(n)//(math.factorial(k)*math.factorial(n-k))
print(C(5,3))
A_likelihood = C(trials,successes)*theta**successes * (1-theta)**(trials - successes)
print(f"probability of observing this data with coin A {A_likelihood}")
# θB
theta = θB_init
B_likelihood = C(trials,successes)*theta**successes * (1-theta)**(trials - successes)
print(f"probability of observing this data with coin B {B_likelihood}")

10
probability of observing this data with coin A 0.20065812480000003
probability of observing this data with coin B 0.24609375


If you refer back to the figure, you'll notice that these values aren't the values assigned to coin A and coin B for trial one. Those values are:

![](trial_1_probabilities.png)

Have we done something wrong, or are we simply not quite there? I wasn't sure at first, but let's make an intuitive guess. In general, statistical tests always require that your total probability sums to 1. Let's try normalizing our data. Given that we have the probability that this data was generated by fictional coin A and the probability that this data was generated by fictional coin B, we can calculate the total probability that this data was generated by either coin

In [20]:
total_likelihood = A_likelihood + B_likelihood

This still doesn't sum to 1, but if we normalize our data to this total, then our updated probabilities, which will now be relative to the total probability, should sum to 1

In [21]:
A_likelihood = A_likelihood / total_likelihood
B_likelihood = B_likelihood / total_likelihood
print(f"{A_likelihood} x A {B_likelihood} x B")

0.4491489261009365 x A 0.5508510738990636 x B


That looks better! And sum to confirm

In [22]:
A_likelihood + B_likelihood

1.0

and with simple rounding we arrive at the same numbers as the authors

In [23]:
print(f"{round(A_likelihood,2)} x A {round(B_likelihood,2)} x B")

0.45 x A 0.55 x B


And we match!
![](trial_1_probabilities.png)

Now that we have calculated the relative likelihood tha each of the two made up coins produced the observed data, let's go ahead and assume that the two made up coins actually did generate this data. We will say that each of the two coins contributed `h` heads and `t` tails. To arrive at this number for each coin, simply multiply the relative probability of each coin by the actual observed numbers for heads and tails of the data

In [24]:
A_heads = int(Data.loc[0, ['HEADS']].values) * A_likelihood;
A_tails = int(Data.loc[0, ['TAILS']].values) * A_likelihood;
print(f'≈{A_heads}H,{A_tails}T')
B_heads = int(Data.loc[0, ['HEADS']].values) * B_likelihood;
B_tails = int(Data.loc[0, ['TAILS']].values) * B_likelihood;
print(f'≈{B_heads}H,{B_tails}T')

≈2.2457446305046824H,2.2457446305046824T
≈2.754255369495318H,2.754255369495318T


And format the values in a DataFrame for easier viewing

![](trial_1_heads_tails_relative_blame.png)

Now we are correctly computing the relative contribution of heads and tails for each coin in terms of generating the observed data. 

This was originally very unintuitive to me, because we know from the original data which coin was actually used to generate each data set. Why would we split responsibility across the two coins? That is a mathematical requirement of the optimization I don't understand. However we can confirm that this is the right approach. [Here is an overview video of the expectation maximization algorithm by Pavel Pevzner from UCSD](https://youtu.be/P1r4RR1goIU?list=PLQ-85lQlPqFMcC2d2CkvmdcJt2v-Np7Cz). I recommend watching the whole playlist of videos, but for the purpose of this tutorial, it's probabily sufficient to watch only the video directly linked to

Now let's write a function that computes the relative responsilibity of a generic row with two generic probabilities. This could be extended to accomidate any number of coins though by passing a list of θ's rather than θ1, θ2

In [25]:
import numpy as np
len(Data)
sigma=Data['HEADS'] + Data['TAILS']
sigma=np.reshape(sigma.values,(len(sigma.values)))
print(sigma)
mid=Data.loc[:, ['HEADS']].values
mid=np.reshape(mid,(len(mid)))
print(mid[1])

[10 10 10 10 10]
9


And now we will map that function onto each row/trial in our dataframe, using the initial guessed probabilities for each of the two coins.

In [26]:
# 概率计算
def assign_blame(row, θ1, θ2):
    successes = row.loc[:, ['HEADS']].values
    successes = np.reshape(successes,len(successes))
    trials = row.loc[:, ['HEADS']].values + row.loc[:, ['TAILS']].values
    trials = np.reshape(trials,len(trials))
    probabilities = []
    for θ in [θ1, θ2]:
        for i in range(len(trials)):
            probabilities.append(C(trials[i],successes[i]) * θ**successes[i] * (1-θ)**(trials[i] - successes[i]))
    probabilities=np.reshape(probabilities,(2,len(trials)))
    total = sum(probabilities)
    probabilities = [p/total for p in probabilities]
    probabilities = np.array(probabilities)
    return probabilities

# map the generic input x to the fxn assign blame, passing x, the initial θA and intial θB, passing each row of the
# DataFrame Data as the input x
blame = assign_blame(Data, θA_init, θB_init) #blame 相当于γ_ij(gamma)

# and hcat to merge them into a matrix. However, the generated matrix is a 2x5 matrix, and we want a 5x2 matrix
print(blame)
print(len(blame[0]))

[[0.44914893 0.80498552 0.73346716 0.35215613 0.64721512]
 [0.55085107 0.19501448 0.26653284 0.64784387 0.35278488]]
5


And for the sake of clarity, let's format this to match the format shown in the figure

In [32]:
datt=[[0,1],[2,3],[4,5]]
datt=np.array(datt)
print(datt[0]*datt[1])

# print(blame[1])
for i in range(len(blame[0])):
    print(f'{blame[0,i]} x A; {blame[1,i]} x B')


[0 3]
0.4491489261009365 x A; 0.5508510738990636 x B
0.804985517232276 x A; 0.19501448276772407 x B
0.7334671580091432 x A; 0.26653284199085686 x B
0.3521561338462594 x A; 0.6478438661537407 x B
0.6472151158991253 x A; 0.3527848841008746 x B


![](relative_blame.png)

Now that we have the relative responsibilites (blame) of each coin, now we count up how many heads and tails each coin deserves based on the level of blame it has. Then recompute probabilities for each coin

In [28]:
data_h=Data.loc[:, ['HEADS']].values
data_h=np.reshape(data_h,len(data_h))
data_t=Data.loc[:, ['TAILS']].values
data_t=np.reshape(data_t,len(data_t))
print(data_h, data_t)

[5 9 8 4 7] [5 1 2 6 3]


In [29]:
Aheads = data_h * blame[0,:];
Atails = data_t * blame[0,:];
print(f'≈{Aheads}H,{Atails}T')

≈[2.24574463 7.24486966 5.86773726 1.40862454 4.53050581]H,[2.24574463 0.80498552 1.46693432 2.1129368  1.94164535]T


In [30]:
Bheads = data_h * blame[1,:];
Btails = data_t * blame[1,:];
print(f'≈{Bheads}H,{Btails}T')

≈[2.75425537 1.75513034 2.13226274 2.59137546 2.46949419]H,[2.75425537 0.19501448 0.53306568 3.8870632  1.05835465]T


Again, let's clean it up for the sake of example

In [26]:
relative_blame = DataFrame(Coin_A = ["≈$(fmt(".1f", CoinA[i,1])) H, $(fmt(".1f", CoinA[i,2])) T" for i = 1:size(CoinA, 1)],
        Coin_B = ["≈$(fmt(".1f", CoinB[i,1])) H, $(fmt(".1f", CoinB[i,2])) T" for i = 1:size(CoinB, 1)])

,Coin_A,Coin_B
1,"≈2.2 H, 2.2 T","≈2.8 H, 2.8 T"
2,"≈7.2 H, 0.8 T","≈1.8 H, 0.2 T"
3,"≈5.9 H, 1.5 T","≈2.1 H, 0.5 T"
4,"≈1.4 H, 2.1 T","≈2.6 H, 3.9 T"
5,"≈4.5 H, 1.9 T","≈2.5 H, 1.1 T"


![](relative_heads_and_tails.png)

Great! However, if we want to update our guess about the maximum likelihood probability of each of our made up coins, we need to calculate

$$ \hat{\theta}_A = \frac{\text{total heads coin A}}{\text{total heads + total tails coin A}}$$

In [27]:
CoinATotals = "≈$(fmt(".1f", sum(Aheads))) H, $(fmt(".1f", sum(Atails))) T"
CoinBTotals = "≈$(fmt(".1f", sum(Bheads))) H, $(fmt(".1f", sum(Btails))) T"
TotalsRow = DataFrame(Coin_A = CoinATotals, Coin_B = CoinBTotals)
vcat(relative_blame, TotalsRow)

,Coin_A,Coin_B
1,"≈2.2 H, 2.2 T","≈2.8 H, 2.8 T"
2,"≈7.2 H, 0.8 T","≈1.8 H, 0.2 T"
3,"≈5.9 H, 1.5 T","≈2.1 H, 0.5 T"
4,"≈1.4 H, 2.1 T","≈2.6 H, 3.9 T"
5,"≈4.5 H, 1.9 T","≈2.5 H, 1.1 T"
6,"≈21.3 H, 8.6 T","≈11.7 H, 8.4 T"


And again, we match
![](relative_blame_with_totals.png)

We're now all set to compute the values, so let's get on with it

In [33]:
θ_sample=data_h/(data_h+data_t)
print(θ_sample)

[0.5 0.9 0.8 0.4 0.7]


In [34]:
# θA_new = sum(Aheads)/sum(Aheads + Atails)
# θB_new = sum(Bheads)/sum(Bheads + Btails)
θA_new = sum(blame[0]*θ_sample)/sum(blame[0])
θB_new = sum(blame[1]*θ_sample)/sum(blame[1])
print(f"new θA ≈ {θA_new}")
print(f"new θB ≈ {θB_new}")

new θA ≈ 0.713012235400516
new θB ≈ 0.5813393083136625


![](recalculate.png)

That's a match! Now, let's put out thetas into a DataFrame to keep track of our iterations

In [35]:
Thetas = pd.DataFrame({'θA' : [θA_new], 'θB' : [θB_new]})
Thetas

,θA,θB
0,0.713012,0.581339


And there you have it. You've successfully walked through the first iteration of the expectation maximization algorithm as it applies to the example in Figure 1 of Do & Batzoglou 2008!

However, the true power of the expectation maximization algorithm is the result of it's iterative potential. Normally, this algorithm will be run until a certain level of convergence is reached. In practice, this can be set to some level of change, such as when your updated thetas change by less than .001 between iterations. Do & Batzoglou 2008 show final output after 10 iterations, so let's go with that

![](convergence.png)

In [36]:
li=list(range(1,10))
print(li[-1])
Thetas.values[-1][0]

9


0.713012235400516

In [39]:
for i in range(20):
    # returns an array of arrays
    blame = assign_blame(Data, Thetas.values[-1][0], Thetas.values[-1][1])
    
    Aheads = data_h * blame[0,:]
    Atails = data_t * blame[0,:]
    
    Bheads = data_h * blame[1,:]
    Btails = data_t * blame[1,:]
    
#     θA_new = sum(Aheads)/sum(Aheads + Atails)
#     θB_new = sum(Bheads)/sum(Bheads + Btails)
    θA_new = sum(blame[0]*θ_sample)/sum(blame[0])
    θB_new = sum(blame[1]*θ_sample)/sum(blame[1])
    
    Thetas = Thetas.append({'θA':θA_new, 'θB':θB_new},ignore_index=True)


And let's see what our final thetas are after 10 iterations by grabbing θA and θB from the Thetas DataFrame at row 10. Remember we are appending each successive iteration as a new row

In [40]:
Thetas
# println(f"θA 10 ≈ Thetas[10, :θA]))")
# println("θB 10 ≈ $(fmt(".2f", Thetas[10, :θB]))")

,θA,θB
0,0.713012,0.581339
1,0.745292,0.569256
2,0.768099,0.549536
3,0.783165,0.534617
4,0.791055,0.526281
5,0.794533,0.522390
6,0.795929,0.520730
7,0.796466,0.520047
8,0.796668,0.519770
9,0.796744,0.519659


Answers match!

![](tenth_iteration.png)

And that's a walkthrough of the entire Expectation Maximization algorithm as it applies to Figure 1 from Do & Batzoglou 2008

![](Figure1.png)

Next steps:
1. try starting from randomly seeded start probabilities
1. implement a dynamic cut off that terminates the EM iterations when we see reduced improvement to the point that we claim convergence

In [41]:
print(blame)

[[0.10300871 0.95201348 0.84549373 0.03070315 0.6014986 ]
 [0.89699129 0.04798652 0.15450627 0.96929685 0.3985014 ]]


In [46]:
# 分类，判断coin属于A=0或B=1
C_coin=np.argmax(blame, axis=0)
print(C_coin)
Agroup=[]
Bgroup=[]
for i in range(len(C_coin)):
    if C_coin[i]==0:
        Agroup.append(i+1)
    else:
        Bgroup.append(i+1)
Class_data = pd.DataFrame({'Coin A' : [Agroup], 'Coin B' : [Bgroup]})
Class_data

[1 0 0 1 0]


,Coin A,Coin B
0,"[2, 3, 5]","[1, 4]"
